### Simple MD (3D) simulation

In [18]:
import numpy as np
import h5py
from polychrom.starting_conformations import grow_cubic
from polychrom.simulation import Simulation
from polychrom.hdf5_format import HDF5Reporter
import polychrom.forcekits as forcekits
import polychrom.forces as forces
import matplotlib.pyplot as plt

### Class for handling and updating bonds

In [15]:
#### Object for handling bonds
class bondUpdater(object):

    def __init__(self, LEFpositions):
        """
        Initialize a bondUpdater object

        :param LEFpositions: numpy array of extruder positions wrt polymer position
        """
        self.LEFpositions = LEFpositions
        self.curtime  = 0
        self.allBonds = []

    def setParams(self, activeParamDict, inactiveParamDict):
        """
        A method to set parameters for bonds.
        It is a separate method because you may want to have a Simulation object already existing

        :param activeParamDict: a dict (argument:value) of addBond arguments for active bonds
        :param inactiveParamDict:  a dict (argument:value) of addBond arguments for inactive bonds

        """
        self.activeParamDict = activeParamDict
        self.inactiveParamDict = inactiveParamDict


    def setup(self, bondForce,  blocks=100, smcStepsPerBlock=1):
        """
        A method that milks smcTranslocator object
        and creates a set of unique bonds, etc.

        :param bondForce: a bondforce object (new after simulation restart!)
        :param blocks: number of blocks to precalculate
        :param smcStepsPerBlock: number of smcTranslocator steps per block
        :return:
        """


        if len(self.allBonds) != 0:
            raise ValueError("Not all bonds were used; {0} sets left".format(len(self.allBonds)))

        self.bondForce = bondForce # force_dict from simulation object (bondForce obj)

        #precalculating all bonds
        allBonds = []
        
        loaded_positions  = self.LEFpositions[self.curtime : self.curtime+blocks] # Get all extruder positions from curtime to curtime+blocks
        allBonds = [[(int(loaded_positions[i, j, 0]), int(loaded_positions[i, j, 1])) 
                        for j in range(loaded_positions.shape[1])] for i in range(blocks)] # Get all positions for both legs of extruder, i.e. location of the 'bonds'

        self.allBonds = allBonds
        self.uniqueBonds = list(set(sum(allBonds, []))) # unlist the bonds and get unique

        #adding forces and getting bond indices
        self.bondInds = []
        self.curBonds = allBonds.pop(0) 

        for bond in self.uniqueBonds: # Loop thru all bonds
            paramset = self.activeParamDict if (bond in self.curBonds) else self.inactiveParamDict # Determine if bond is active and get parameters
            ind = bondForce.addBond(bond[0], bond[1], **paramset) # 
            self.bondInds.append(ind)
        self.bondToInd = {i:j for i,j in zip(self.uniqueBonds, self.bondInds)} # Dict of {bond : bond index}
        
        self.curtime += blocks # Advance blocks
        
        return self.curBonds,[]


    def step(self, context, verbose=False):
        """
        Update the bonds to the next step.
        It sets bonds for you automatically!
        :param context:  context
        :return: (current bonds, previous step bonds); just for reference
        """
        if len(self.allBonds) == 0:
            raise ValueError("No bonds left to run; you should restart simulation and run setup  again")

        pastBonds = self.curBonds
        self.curBonds = self.allBonds.pop(0)  # getting current bonds
        bondsRemove = [i for i in pastBonds if i not in self.curBonds] # ID bonds to remove
        bondsAdd = [i for i in self.curBonds if i not in pastBonds] # Bonds to add
        bondsStay = [i for i in pastBonds if i in self.curBonds] # Bonds to stay
        if verbose:
            print("{0} bonds stay, {1} new bonds, {2} bonds removed".format(len(bondsStay),
                                                                            len(bondsAdd), len(bondsRemove)))
        bondsToChange = bondsAdd + bondsRemove # Total num of bonds to change
        bondsIsAdd = [True] * len(bondsAdd) + [False] * len(bondsRemove) # Flag for if a bond should be added
        for bond, isAdd in zip(bondsToChange, bondsIsAdd): # loop thru all bonds
            ind = self.bondToInd[bond] # Get index of this bond
            paramset = self.activeParamDict if isAdd else self.inactiveParamDict # Fetch parameters
            self.bondForce.setBondParameters(ind, bond[0], bond[1], **paramset)  # actually updating bonds
        self.bondForce.updateParametersInContext(context)  # now run this to update things in the context
        return self.curBonds, pastBonds

### Define parameters for 3D portion

In [16]:
##### Define parameters
trajectories = h5py.File("../1D_trajectory/trajectory/LEFPositions.h5") # Saved trajectories from 1D siumulation

N = trajectories.attrs["N"] # Length of polymer
LEFNum = trajectories.attrs["LEFNum"] # Number of extruders
LEFpositions = trajectories["positions"] # Positions of extruders at each 1D step
Nframes = LEFpositions.shape[0] # Number of 1D steps (= number of extruder steps)

steps = 100 # MD steps PER STEP OF EXTRUDER
stiff = 1
box = (N / 0.1) ** 0.33 # Dimensions of bounding box
data = grow_cubic(N, int(box))

# SMC (Extruder) parameters
smcBondWiddleDist = 0.2
smcBondDist = 0.5

### Simulation saving parameters
saveEveryBlocks = 10 # Write coordinates every this many blocks
restartSimulationEveryBlocks = 100 # 
# Checks
assert Nframes % restartSimulationEveryBlocks == 0 # So we don't have leftover steps that won't get saved
assert (restartSimulationEveryBlocks % saveEveryBlocks) == 0

savesPerSim = restartSimulationEveryBlocks // saveEveryBlocks
simInitsTotal = Nframes // restartSimulationEveryBlocks # Number of simulation initializations

print("""
There will be {} MD steps done for every step of the extruder
Conformations saved every {} steps, for {} confs. per simulation
Simulation restarts every {} steps
      """.format(steps,saveEveryBlocks,savesPerSim,restartSimulationEveryBlocks))


There will be 100 MD steps done for every step of the extruder
Conformations saved every 10 steps, for 10 confs. per simulation
Simulation restarts every 100 steps
      


### The Simulation Loop

In [20]:
milker = bondUpdater(LEFpositions)

reporter = HDF5Reporter(folder="sim_outs", # Save data location
                        max_data_length=100, # Write data in chunks of this size
                        overwrite=True, # overwrite existing file in out location
                        blocks_only=True) # only save simulation blocks

for iter in range(simInitsTotal):
    # Create the simulation object
    a = Simulation(
            platform="cuda", # platform to do computations on
            integrator="variableLangevin", # Integrator from OpenMM
            error_tol=0.01, # error rate parameter for variableLangevin integrator
            GPU="0", # GPU index
            collision_rate=0.03, # collision rate of particles in inverse picoseconds
            N=len(data), # no. of particles
            reporters=[reporter], # list of reporter objects to use
            PBCbox=[box,box,box], # Periodic Boundary Conditions (PBC) box dimensions (x,y,z)
            precision="mixed" # GPU calculation precision, mixed is slow on 3080 and newer GPUs
    )
    # Loads the polymer we created, and puts center of mass at (0,0,0)
    a.set_data(data) 
    # Add a force to the simulation object - since we are doing polymer simulation, we add a 'forcekit' that describes all the forces in a polymer chain and the interactions between them
    a.add_force(
        forcekits.polymer_chains(
            a, # Simulation object
            chains=[(0, None, 0)], # List of tuples desctibing 1 chain each - this is the default value, i.e. one chain of length N that is not a ring (i.e. a chain)
            bond_force_func=forces.harmonic_bonds, # Define the bonded force as harmonic bonds
            bond_force_kwargs={'bondLength':1.0, 'bondWiggleDistance':0.1}, # Parameters for harmonic bonds
            angle_force_func=forces.angle_force, # Angle force 
            angle_force_kwargs={'k':1.5}, # Angle force parameters. k = stiffness bond (8=very stiff, k=1.5 is "realistically flexible")
            nonbonded_force_func=forces.polynomial_repulsive, # Nonbonded force
            nonbonded_force_kwargs={'trunc':1.5, # Allows chains to cross, the energy value at dist=0
                                    'radiusMult':1.05},
            except_bonds=True # Monomers that are directly bonded will not take into account nonbonded forces
        )
    )
    # Calculate bond parameters for extruder contact
    kbond = a.kbondScalingFactor / (smcBondWiddleDist**2)
    bondDist = smcBondDist * a.length_scale
    activeParams = {"length":bondDist, "k":kbond}
    inactiveParams = {"length":bondDist, "k":0}
    # Set up bond manager object ("milker")
    milker.setParams(activeParams, inactiveParams)
    milker.setup(bondForce=a.force_dict["harmonic_bonds"], blocks=restartSimulationEveryBlocks)

    # During the first simulation initiation, minimize energy of conformations
    if iter == 0:
        a.local_energy_minimization()
    else:
        a._apply_forces()
    for i in range(restartSimulationEveryBlocks):
        if i % saveEveryBlocks == (saveEveryBlocks-1):
            a.do_block(steps=steps) # do steps and get new monomer positions
        else:
            a.integrator.step(steps) # do steps WITHOUT getting new monomer positions (faster)
        if i < restartSimulationEveryBlocks - 1:
            curBonds, pastBonds = milker.step(a.context) # Update bonds with the milker
    data = a.get_data() # Fetch new polymer positions 
    del a 

    reporter.blocks_only = True # Write only blocks, not individual steps in block
    time.sleep(0.2) # wait to garbage collector can clean up

reporter.dump_data() # Output

INFO:root:Performing local energy minimization
INFO:root:adding force harmonic_bonds 0
INFO:root:adding force angle 1
INFO:root:Using periodic boundary conditions
INFO:root:adding force polynomial_repulsive 2


Exclude neighbouring chain particles from polynomial_repulsive
Number of exceptions: 499


OpenMMException: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1706893044997/work/platforms/cuda/src/CudaContext.cpp:91